In [1]:
# Standard libraries of Python

from time import gmtime

# Dependencies
import pandas as pd
import requests
import re
from pandas import DataFrame
from bs4 import BeautifulSoup

from backend.database.src import parse

In [46]:
html_text = requests.get(f'https://www.serebii.net/pokedex-sm/001.shtml').text
soup = BeautifulSoup(html_text, 'html.parser')

In [47]:
all_divs = soup.find_all('div', attrs={'align': 'center'})

In [48]:
all_divs

[<div align="center"><table align="center" border="0" cellpadding="4" cellspacing="0" width="95%">
 <tr>
 <td align="center" width="25%"><form name="nav">
 <select name="SelectURL" onchange="document.location.href=document.nav.SelectURL.options[document.nav.SelectURL.selectedIndex].value" size="1" style="color:#383838; font-size: 8pt; background:#CEBC77">
 <option value="#">Kanto: 001 - 151</option>
 <option value="/pokedex-sm/001.shtml">001 Bulbasaur</option>
 <option value="/pokedex-sm/002.shtml">002 Ivysaur</option>
 <option value="/pokedex-sm/003.shtml">003 Venusaur</option>
 <option value="/pokedex-sm/004.shtml">004 Charmander</option>
 <option value="/pokedex-sm/005.shtml">005 Charmeleon</option>
 <option value="/pokedex-sm/006.shtml">006 Charizard</option>
 <option value="/pokedex-sm/007.shtml">007 Squirtle</option>
 <option value="/pokedex-sm/008.shtml">008 Wartortle</option>
 <option value="/pokedex-sm/009.shtml">009 Blastoise</option>
 <option value="/pokedex-sm/010.shtml">01

In [183]:
ala = all_divs[0].find_all('td', {'class': 'fooinfo'})

[<td align="center" class="fooinfo"><table><tr><td class="pkmn"><img alt="Normal Sprite" src="/sunmoon/pokemon/001.png"/></td><td class="pkmn"><img alt="Shiny Sprite" src="/Shiny/SM/001.png"/></td></tr></table></td>,
 <td class="fooinfo">Bulbasaur</td>,
 <td class="fooinfo"><table cellpadding="0" cellspacing="0" width="120">
 <tr> <td><b>Japan</b>: </td><td>Fushigidane<br/>フシギダネ</td></tr>
 <tr> <td><b>French</b>: </td><td>Bulbizarre</td></tr>
 <tr> <td><b>German</b>: </td><td>Bisasam</td></tr>
 <tr> <td><b>Korean</b>: </td><td>이상해씨</td></tr>
 </table></td>,
 <td class="fooinfo"><table cellpadding="0" cellspacing="0" width="120"><tr> <td><b>National</b>: </td><td>#001</td></tr>
 <tr><td><b>Alola</b>: </td><td>#---</td></tr>
 <tr><td><b>Melemele</b>: </td><td>#---</td></tr>
 <tr><td><b>Akala</b>: </td><td>#---</td></tr>
 <tr><td><b>Ula'ula</b>: </td><td>#---</td></tr>
 <tr><td><b>Poni</b>: </td><td>#---</td></tr><tr><td><b>Kanto</b>: </td><td>#001</td></tr>
 </table></td>,
 <td class="fo

In [298]:
tags = ala[10].find_all('b')

In [308]:
for tag in tags:
    ability = tag.string

    if ability == 'Hidden Ability':
        continue
    
    ability_text = ability.next_element.strip()

    x = [ability,ability_text]
    x = ''.join(x)

    print(x)

Overgrow: When HP is below 1/3rd its maximum, power of Grass-type moves is increased by 50%.
Chlorophyll: When sunny, the Pokémon’s Speed doubles. However, Speed will not double on the turn weather becomes Strong Sunlight.


In [354]:
f = ala[15].find('form', {'name': 'breed'}).find_all('option')

In [355]:
text = [p.text for p in f]
print(text)

['Monster', 'Bulbasaur', 'Ivysaur', 'Venusaur', 'Charmander', 'Charmeleon', 'Charizard', 'Squirtle', 'Wartortle', 'Blastoise', 'Nidoran♀', 'Nidoran♂', 'Nidorino', 'Nidoking', 'Slowpoke', 'Slowbro', 'Cubone', 'Marowak', 'Lickitung', 'Rhyhorn', 'Rhydon', 'Kangaskhan', 'Lapras', 'Snorlax', 'Chikorita', 'Bayleef', 'Meganium', 'Totodile', 'Croconaw', 'Feraligatr', 'Mareep', 'Flaaffy', 'Ampharos', 'Slowking', 'Larvitar', 'Pupitar', 'Tyranitar', 'Treecko', 'Grovyle', 'Sceptile', 'Mudkip', 'Marshtomp', 'Swampert', 'Whismur', 'Loudred', 'Exploud', 'Aron', 'Lairon', 'Aggron', 'Tropius', 'Turtwig', 'Grotle', 'Torterra', 'Cranidos', 'Rampardos', 'Shieldon', 'Bastiodon', 'Gible', 'Gabite', 'Garchomp', 'Snover', 'Abomasnow', 'Lickilicky', 'Rhyperior', 'Axew', 'Fraxure', 'Haxorus', 'Druddigon', 'Helioptile', 'Heliolisk', 'Tyrunt', 'Tyrantrum', 'Amaura', 'Aurorus', 'Bergmite', 'Avalugg', 'Salandit', 'Salazzle', 'Turtonator', 'Drampa', 'Ditto']


In [358]:
len(ala[15].find_all('form'))

2

In [ ]:
scrap = parse.Dextable(2)

In [ ]:
scrap.get_gender()

In [ ]:
scrap._name

In [ ]:
name = scrap.soup.find('title').text.split()
text_to_eliminate = '-','','Serebii.net','Pokédex'
cleaned_name = [element for element in name if element not in text_to_eliminate]
name,number = cleaned_name[0],cleaned_name[1]
print(name)
print(number)

In [ ]:
number = int(re.sub(r'#', '', number))

In [ ]:
name

In [ ]:
self._name, self._number = cleaned_name[0], cleaned_name[1]
self._number = int(re.sub(r'#', '', self._number[1]))

In [ ]:
pokedex = pd.DataFrame()

In [ ]:
assert requests.get("https://www.serebii.net/index2.shtml").status_code == 200, 'There is a problem with Euromillions webpage'

In [ ]:
soup = BeautifulSoup(requests.get('https://www.serebii.net/pokedex-sm/001.shtml').text, 'html.parser')

In [ ]:
soup

In [ ]:
dex = soup.find_all('table', class_ = 'dextable')

In [ ]:
ala = dex[1]

In [ ]:
dex[1]